In [ ]:
# Install PyTorch with GPU (CUDA) support
# For CUDA 12.1 (most common). For other versions, see https://pytorch.org
import subprocess
import sys

subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "torch", "torchvision", "torchaudio", "--index-url", "https://download.pytorch.org/whl/cu121"])
print("PyTorch with CUDA 12.1 installed successfully!")

In [9]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [13]:
import torch
print(torch.cuda.is_available())
# If installed correctly, this should return True

False


Cell 1: Setup & Device Check

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split, Dataset
import os

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if device.type == 'cpu':
    print("⚠️ WARNING: You are training on CPU. This will be slow.")
    print("If you have an NVIDIA GPU, make sure you installed the correct PyTorch version with CUDA support.")

Using device: cpu
⚠️ WARNING: You are training on CPU. This will be slow.
If you have an NVIDIA GPU, make sure you installed the correct PyTorch version with CUDA support.


Cell 2: Fixed Data Loading (The Important Fix)


In [15]:
# 1. Define Transforms (Resized to 224 for speed & accuracy)
IMG_SIZE = 224 

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(), # Augmentation only for training
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# 2. Helper Class to fix the "Split" bug
class MapDataset(Dataset):
    def __init__(self, dataset, map_transform):
        self.dataset = dataset
        self.map_transform = map_transform

    def __getitem__(self, index):
        if self.map_transform:
            x = self.dataset[index][0] 
            return self.map_transform(x), self.dataset[index][1]
        else:
            return self.dataset[index]

    def __len__(self):
        return len(self.dataset)

# 3. Load Public_dataset
data_dir = 'Public_dataset' # Ensure this folder exists
# Load raw dataset first
full_dataset = datasets.ImageFolder(data_dir)

# 4. Split indices
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_subset, val_subset = random_split(full_dataset, [train_size, val_size])

# 5. Apply transforms CORRECTLY using the helper class
train_dataset = MapDataset(train_subset, data_transforms['train'])
val_dataset = MapDataset(val_subset, data_transforms['val'])

# 6. Create DataLoaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
}

class_names = full_dataset.classes
NUM_CLASSES = len(class_names)
print(f"Classes: {class_names}")
print(f"Training samples: {len(train_dataset)} | Validation samples: {len(val_dataset)}")

Classes: ['aluminium', 'paper', 'plastic']
Training samples: 2147 | Validation samples: 537


Cell 3: Model Setup

In [16]:
# Load Pre-trained ResNet50
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

# Freeze all layers (so we don't destroy pre-trained features)
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer for our 3 classes (Aluminium, Paper, Plastic)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)

model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
# Optimizing ONLY the final layer (model.fc)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

Cell 4: Training Loop

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# Train
model = train_model(model, dataloaders, criterion, optimizer, num_epochs=10)

# Save the model
torch.save(model.state_dict(), 'resnet50_waste_fixed.pth')
print("Model saved successfully!")

Epoch 1/10
----------


Cell 5: Final Test on "Dataset"

from sklearn.metrics import classification_report, confusion_matrix

# 1. Load the separate Test Dataset
test_dir = 'Dataset' # The folder you want to use for final testing
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['val'])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Testing on {len(test_dataset)} images from '{test_dir}'...")

# 2. Evaluate
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 3. Print Results
print("\nFinal Test Results:")
print(classification_report(all_labels, all_preds, target_names=class_names))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))